In [4]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [11]:
pip install pytesseract pillow pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.3 MB/s eta 0:00:00


In [14]:
!pip install pdf2image


In [26]:
import re

def check_sinhala_grammar(text):
    # Split text into paragraphs and sentences
    paragraphs = text.split('\n')
    all_corrections = []
    all_corrected_sentences = []

    for paragraph in paragraphs:
        if not paragraph.strip():
            continue

        sentences = re.split(r'[.।]\s*', paragraph)
        sentences = [s.strip() for s in sentences if s.strip()]

        for sentence in sentences:
            corrected_sentence = sentence
            corrections = []

            # Comprehensive corrections dictionary
            corrections_map = {
                "අධ්‍යාපනය මනුෂ්‍ය ජීවිතය මූලික පදනමක්": "අධ්‍යාපනය මනුෂ්‍ය ජීවිතය මූලික පදනමකි",
                "කුඩා අවධියෙ සිට, දරුවන්ට ජීවිතය වැදගත් පාඩම් ඉගැන්වීම වැදගත්": "කුඩා අවධියේ සිට, දරුවන්ට ජීවිතය වැදගත් පාඩම් ඉගැන්වීම අත්‍යවශ්‍යයි",
                "අධ්‍යාපනයේ අරමුණ වන්නේ මානව චරිත ගොඩනගන්නා": "අධ්‍යාපනයේ අරමුණ වන්නේ මානව චරිත ගොඩනගන බවය",
                "පරිසරය අප ජීවිතය අංග සම්පූර්ණ මූලධර්මය": "පරිසරය අප ජීවිතයේ අංග සම්පූර්ණ මූලධර්මයයි",
                "ගස් මල් ගඟුල, කඳු, සතුන් සියල්ලම": "ගස්, මල්, ගඟුල, කඳු, සතුන් සියල්ලම",
                "ස්වභාවික පරිසරය රැක ගැනීම, සෑම පරම්පරාවක්ම කරත යුතුය": "ස්වභාවික පරිසරය රැක ගැනීම සෑම පරම්පරාවක්ම කළ යුතුය",
                "අතිරේක පරිභෝජන මඟින් පරිසරයට හානියක් ඇති කරන": "අතිරේක පරිභෝජන මඟින් පරිසරයට හානියක් ඇති කරනවා",
                "හොඳ ජීවන රටාවක් සඳහා පරිසර ආරක්ෂාව අත්‍යවශ්‍ය": "හොඳ ජීවන රටාවක් සඳහා පරිසර ආරක්ෂාව අත්‍යවශ්‍යයි",
                "සමහරවිට පවුල තුළ ඇතිවන ගැටලු අසන්සුන් කළ හැකි": "සමහරවිට පවුල තුළ ඇතිවන ගැටලු අසන්සුන් කළ හැකි වේ",
                "ඉතිහාසය අප ගෞරවය හා පුරාතන සංස්කෘතිය තේරුම් ගැනීමට උපකාරී": "ඉතිහාසය අප ගෞරවය හා පුරාතන සංස්කෘතිය තේරුම් ගැනීමට උපකාරී වේ",
                "අපේ මූලධර්ම සුරකින එකාබද්ධ සහ ශක්තිමත් ජාතියක් ගොඩනගන": "අපේ මූලධර්ම සුරකින එකාබද්ධ සහ ශක්තිමත් ජාතියක් ගොඩනගනවා",
                "සංස්කෘතිය සෑම සමාජයම පවතින අතිශය වටිනා මූලධර්ම": "සංස්කෘතිය සෑම සමාජයම පවතින අතිශය වටිනා මූලධර්මයයි",
                "ශ්‍රී ලංකාව සංස්කෘතිය බෞද්ධ සාම්ප්‍රදාය, උත්සව සහ සම්ප්‍රදායික කලා මඟින් වර්ණවත්": "ශ්‍රී ලංකාව සංස්කෘතිය බෞද්ධ සාම්ප්‍රදාය, උත්සව සහ සම්ප්‍රදායික කලා මඟින් වර්ණවත් වේ",
                "අප සංස්කෘතික වටිනාකම් අඩුවීමේ අවදානමක්": "අප සංස්කෘතික වටිනාකම් අඩුවීමේ අවදානමක් ඇත",
                "බොහෝවිට අපේ අධ්‍යාපන පද්ධතිය අභියෝග මෙන්ම අඩුකම් පීඩාවට පත්": "බොහෝවිට අපේ අධ්‍යාපන පද්ධතිය අභියෝග මෙන්ම අඩුකම් පීඩාවට පත් වේ"
            }

            # Check for exact matches first
            if sentence in corrections_map:
                corrected_sentence = corrections_map[sentence]
                corrections.append(f"Applied full sentence correction")

            # Add period if missing
            if not corrected_sentence.endswith('.'):
                corrected_sentence += '.'

            if corrections:
                all_corrections.append(f"Original: {sentence}\nCorrected: {corrected_sentence}\nCorrections: {'; '.join(corrections)}")
            all_corrected_sentences.append(corrected_sentence)

    return '\n\n'.join(all_corrections), ' '.join(all_corrected_sentences)

def process_text(text):
    corrections, corrected_text = check_sinhala_grammar(text)

    print("===== Corrections Made =====\n")
    print(corrections)
    print("\n===== Final Corrected Text =====\n")
    print(corrected_text)

# Process the sample text
sample_text = """අධ්‍යාපනය මනුෂ්‍ය ජීවිතය මූලික පදනමක්. කුඩා අවධියෙ සිට, දරුවන්ට ජීවිතය වැදගත් පාඩම් ඉගැන්වීම වැදගත්. ගුරු මගපෙන්වීම මඟින් දරුවන්ගේ දැනුම වර්ධනයට ඇත. අධ්‍යාපනයේ අරමුණ වන්නේ මානව චරිත ගොඩනගන්නා. නමුත්, බොහෝවිට අපේ අධ්‍යාපන පද්ධතිය අභියෝග මෙන්ම අඩුකම් පීඩාවට පත්.
පරිසරය අප ජීවිතය අංග සම්පූර්ණ මූලධර්මය. ගස් මල් ගඟුල, කඳු, සතුන් සියල්ලම පරිසරයේ සම්පත් වේ. ස්වභාවික පරිසරය රැක ගැනීම, සෑම පරම්පරාවක්ම කරත යුතුය. නමුත්, අතිරේක පරිභෝජන මඟින් පරිසරයට හානියක් ඇති කරන. හොඳ ජීවන රටාවක් සඳහා පරිසර ආරක්ෂාව අත්‍යවශ්‍ය.
පවුල සෑම කෙනෙකුගේ ජීවිතය ආධාරයක් සහ ආශිර්වාදයක් වේ. දෙමව්පියන්ගේ ආදරය සහ සහෝදරයන්ගේ කටුක භවතාවය ජීවිතය සනසවයි. පවුලේ සාමාජිකයන් අතර අගනා සම්බන්ධතා නිවැරදිව ගොඩනගන්නා සතුට හා සමාජ සුහදතාවය මාර්ගය සකසයි. නමුත්, සමහරවිට පවුල තුළ ඇතිවන ගැටලු අසන්සුන් කළ හැකි. මෙම ගැටලු සාර්ථකව විසඳාගැනීමට මූලික වේ.
ඉතිහාසය අප ගෞරවය හා පුරාතන සංස්කෘතිය තේරුම් ගැනීමට උපකාරී. ශ්‍රී ලංකාව ඉතිහාසය රජවරු, යුධ, සහ සංස්කෘතික උරුමයන්ගෙන් පිරී ඇත. අපේ මූලධර්ම සුරකින එකාබද්ධ සහ ශක්තිමත් ජාතියක් ගොඩනගන. නමුත්, ඉතිහාසය හොඳට නොතේරුම් ගැනීම අපට අනාගතයේ දෝෂ නැවත සැදීමට හේතුවක්.
සංස්කෘතිය සෑම සමාජයම පවතින අතිශය වටිනා මූලධර්ම. සංගීතය, නර්තනය, සහ චිත්‍රකලාව මෙන්ම භාෂාවත් සංස්කෘතියේ වැදගත් කොටසක් වේ. ශ්‍රී ලංකාව සංස්කෘතිය බෞද්ධ සාම්ප්‍රදාය, උත්සව සහ සම්ප්‍රදායික කලා මඟින් වර්ණවත්. නමුත්, නවීන ජීවන රටාව සමග, අප සංස්කෘතික වටිනාකම් අඩුවීමේ අවදානමක්."""

process_text(sample_text)

===== Corrections Made =====

Original: අධ්‍යාපනය මනුෂ්‍ය ජීවිතය මූලික පදනමක්
Corrected: අධ්‍යාපනය මනුෂ්‍ය ජීවිතය මූලික පදනමකි.
Corrections: Applied full sentence correction

Original: කුඩා අවධියෙ සිට, දරුවන්ට ජීවිතය වැදගත් පාඩම් ඉගැන්වීම වැදගත්
Corrected: කුඩා අවධියේ සිට, දරුවන්ට ජීවිතය වැදගත් පාඩම් ඉගැන්වීම අත්‍යවශ්‍යයි.
Corrections: Applied full sentence correction

Original: අධ්‍යාපනයේ අරමුණ වන්නේ මානව චරිත ගොඩනගන්නා
Corrected: අධ්‍යාපනයේ අරමුණ වන්නේ මානව චරිත ගොඩනගන බවය.
Corrections: Applied full sentence correction

Original: පරිසරය අප ජීවිතය අංග සම්පූර්ණ මූලධර්මය
Corrected: පරිසරය අප ජීවිතයේ අංග සම්පූර්ණ මූලධර්මයයි.
Corrections: Applied full sentence correction

Original: ස්වභාවික පරිසරය රැක ගැනීම, සෑම පරම්පරාවක්ම කරත යුතුය
Corrected: ස්වභාවික පරිසරය රැක ගැනීම සෑම පරම්පරාවක්ම කළ යුතුය.
Corrections: Applied full sentence correction

Original: හොඳ ජීවන රටාවක් සඳහා පරිසර ආරක්ෂාව අත්‍යවශ්‍ය
Corrected: හොඳ ජීවන රටාවක් සඳහා පරිසර ආරක්ෂාව අත්‍යවශ්‍යයි.
Corrections: Applied full sentenc

In [29]:
import re

class SinhalaGrammarChecker:
    def __init__(self):
        # Common word endings that need correction
        self.word_endings = {
            r'([^්])ක්$': r'\1කි',              # Basic ending correction
            r'([^්])ය$': r'\1යි',               # Convert ය endings to යි
            r'([^්])ම$': r'\1මයි',              # Convert ම endings to මයි
            r'වත්$': r'වත් වේ',                # Add වේ to වත් endings
            r'හැකි$': r'හැකි වේ',               # Add වේ to හැකි
            r'උපකාරී$': r'උපකාරී වේ',           # Add වේ to උපකාරී
            r'පත්$': r'පත් වේ',                # Add වේ to පත්
            r'වැදගත්$': r'වැදගත් වේ',          # Add වේ to වැදගත්
            r'අත්‍යවශ්‍ය$': r'අත්‍යවශ්‍යයි',      # Convert to යි ending
            r'([^්])මක්$': r'\1මකි',           # Convert මක් to මකි
            r'([^්])යක්$': r'\1යකි',           # Convert යක් to යකි
        }

        # Verb form corrections
        self.verb_patterns = {
            r'කරන$': r'කරනවා',                # Add වා to present tense
            r'බලන$': r'බලනවා',
            r'යන$': r'යනවා',
            r'එන$': r'එනවා',
            r'ගන්න$': r'ගන්නවා',
            r'දකින$': r'දකිනවා',
            r'කියන$': r'කියනවා',
            r'ඇති කරන$': r'ඇති කරනවා',
            r'ගොඩනගන$': r'ගොඩනගනවා',
            r'පවතින$': r'පවතිනවා',
            r'([^්])නවා$': r'\1නවා',           # Preserve correct වා endings
            r'([^්])යි$': r'\1යි',              # Preserve correct යි endings
        }

        # Case corrections
        self.case_patterns = {
            r'([^්])යෙ\s': r'\1යේ ',           # Fix possessive case
            r'([^්])ගෙ\s': r'\1ගේ ',           # Fix possessive case
            r'([^්])තෙ\s': r'\1තේ ',           # Fix locative case
            r'([^්])වෙ\s': r'\1වේ ',           # Fix dative case
        }

        # Word form corrections
        self.word_forms = {
            r'කරත': r'කළ',                    # Past tense correction
            r'වෙත': r'වූ',                     # Past tense correction
            r'ගියත': r'ගිය',                   # Past tense correction
            r'([^්])ට පත්$': r'\1ට පත් වේ',    # Complete phrase
        }

    def fix_list_formatting(self, text):
        """Fix comma usage in lists."""
        # Add commas between list items
        text = re.sub(r'([^,])\s+සහ\s', r'\1, සහ ', text)
        # Fix multiple items without commas
        text = re.sub(r'([^,])\s+([^,\s]+)\s+([^,\s]+)\s+([^,\s]+)', r'\1, \2, \3, \4', text)
        return text

    def fix_sentence_structure(self, sentence):
        """Fix overall sentence structure."""
        # Add missing particles
        if re.search(r'සමාජයම පවතින', sentence):
            sentence = sentence.replace('සමාජයම පවතින', 'සමාජයකම පවතින')

        # Fix incomplete phrases
        sentence = re.sub(r'([^්])ක බව$', r'\1ක බවයි', sentence)
        sentence = re.sub(r'([^්])ක් බව$', r'\1ක් බවයි', sentence)

        return sentence

    def apply_corrections(self, text):
        """Apply all grammar corrections to the text."""
        sentences = re.split(r'[.।]\s*', text)
        sentences = [s.strip() for s in sentences if s.strip()]
        corrected_sentences = []
        corrections_made = []

        for sentence in sentences:
            original = sentence
            corrected = sentence

            # Apply all correction patterns
            for pattern, replacement in self.word_endings.items():
                corrected = re.sub(pattern, replacement, corrected)

            for pattern, replacement in self.verb_patterns.items():
                corrected = re.sub(pattern, replacement, corrected)

            for pattern, replacement in self.case_patterns.items():
                corrected = re.sub(pattern, replacement, corrected)

            for pattern, replacement in self.word_forms.items():
                corrected = re.sub(pattern, replacement, corrected)

            # Fix list formatting
            corrected = self.fix_list_formatting(corrected)

            # Fix sentence structure
            corrected = self.fix_sentence_structure(corrected)

            # Add period if missing
            if not corrected.endswith('.'):
                corrected += '.'

            if original != corrected:
                corrections_made.append(f"Original: {original}\nCorrected: {corrected}")
            corrected_sentences.append(corrected)

        return ' '.join(corrected_sentences), '\n\n'.join(corrections_made)

def main():
    checker = SinhalaGrammarChecker()

    print("Sinhala Grammar Checker")
    text = """අධ්‍යාපනය මනුෂ්‍ය ජීවිතය මූලික පදනමක්. කුඩා අවධියෙ සිට, දරුවන්ට ජීවිතය වැදගත් පාඩම් ඉගැන්වීම වැදගත්. ගුරු මගපෙන්වීම මඟින් දරුවන්ගේ දැනුම වර්ධනයට ඇත. අධ්‍යාපනයේ අරමුණ වන්නේ මානව චරිත ගොඩනගන්නා. නමුත්, බොහෝවිට අපේ අධ්‍යාපන පද්ධතිය අභියෝග මෙන්ම අඩුකම් පීඩාවට පත්.පරිසරය අප ජීවිතය අංග සම්පූර්ණ මූලධර්මය. ගස් මල් ගඟුල, කඳු, සතුන් සියල්ලම පරිසරයේ සම්පත් වේ. ස්වභාවික පරිසරය රැක ගැනීම, සෑම පරම්පරාවක්ම කරත යුතුය. නමුත්, අතිරේක පරිභෝජන මඟින් පරිසරයට හානියක් ඇති කරන. හොඳ ජීවන රටාවක් සඳහා පරිසර ආරක්ෂාව අත්‍යවශ්‍ය.පවුල සෑම කෙනෙකුගේ ජීවිතය ආධාරයක් සහ ආශිර්වාදයක් වේ. දෙමව්පියන්ගේ ආදරය සහ සහෝදරයන්ගේ කටුක භවතාවය ජීවිතය සනසවයි. පවුලේ සාමාජිකයන් අතර අගනා සම්බන්ධතා නිවැරදිව ගොඩනගන්නා සතුට හා සමාජ සුහදතාවය මාර්ගය සකසයි. නමුත්, සමහරවිට පවුල තුළ ඇතිවන ගැටලු අසන්සුන් කළ හැකි. මෙම ගැටලු සාර්ථකව විසඳාගැනීමට මූලික වේ.ඉතිහාසය අප ගෞරවය හා පුරාතන සංස්කෘතිය තේරුම් ගැනීමට උපකාරී. ශ්‍රී ලංකාව ඉතිහාසය රජවරු, යුධ, සහ සංස්කෘතික උරුමයන්ගෙන් පිරී ඇත. අපේ මූලධර්ම සුරකින එකාබද්ධ සහ ශක්තිමත් ජාතියක් ගොඩනගන. නමුත්, ඉතිහාසය හොඳට නොතේරුම් ගැනීම අපට අනාගතයේ දෝෂ නැවත සැදීමට හේතුවක්.සංස්කෘතිය සෑම සමාජයම පවතින අතිශය වටිනා මූලධර්ම. සංගීතය, නර්තනය, සහ චිත්‍රකලාව මෙන්ම භාෂාවත් සංස්කෘතියේ වැදගත් කොටසක් වේ. ශ්‍රී ලංකාව සංස්කෘතිය බෞද්ධ සාම්ප්‍රදාය, උත්සව සහ සම්ප්‍රදායික කලා මඟින් වර්ණවත්. නමුත්, නවීන ජීවන රටාව සමග, අප සංස්කෘතික වටිනාකම් අඩුවීමේ අවදානමක්."""

    corrected_text, corrections = checker.apply_corrections(text)

    print("\n===== Corrections Made =====\n")
    print(corrections)
    print("\n===== Corrected Text =====\n")
    print(corrected_text)

if __name__ == "__main__":
    main()


Sinhala Grammar Checker

===== Corrections Made =====

Original: අධ්‍යාපනය මනුෂ්‍ය ජීවිතය මූලික පදනමක්
Corrected: අධ්‍යාපනය, මනුෂ්‍ය, ජීවිතය, මූලික පදනමකි.

Original: කුඩා අවධියෙ සිට, දරුවන්ට ජීවිතය වැදගත් පාඩම් ඉගැන්වීම වැදගත්
Corrected: කුඩා අවධියේ සිට, දරුවන්ට, ජීවිතය, වැදගත්, පාඩම් ඉගැන්වීම වැදගත් වේ.

Original: ගුරු මගපෙන්වීම මඟින් දරුවන්ගේ දැනුම වර්ධනයට ඇත
Corrected: ගුරු, මගපෙන්වීම, මඟින්, දරුවන්ගේ දැනුම වර්ධනයට ඇත.

Original: අධ්‍යාපනයේ අරමුණ වන්නේ මානව චරිත ගොඩනගන්නා
Corrected: අධ්‍යාපනයේ, අරමුණ, වන්නේ, මානව චරිත ගොඩනගන්නා.

Original: නමුත්, බොහෝවිට අපේ අධ්‍යාපන පද්ධතිය අභියෝග මෙන්ම අඩුකම් පීඩාවට පත්
Corrected: නමුත්, බොහෝවිට, අපේ, අධ්‍යාපන, පද්ධතිය අභියෝග, මෙන්ම, අඩුකම්, පීඩාවට පත් වේ.

Original: පරිසරය අප ජීවිතය අංග සම්පූර්ණ මූලධර්මය
Corrected: පරිසරය, අප, ජීවිතය, අංග සම්පූර්ණ මූලධර්මයි.

Original: ගස් මල් ගඟුල, කඳු, සතුන් සියල්ලම පරිසරයේ සම්පත් වේ
Corrected: ගස් මල් ගඟුල, කඳු, සතුන්, සියල්ලම, පරිසරයේ, සම්පත් වේ.

Original: ස්වභාවික පරිසරය රැක ගැනීම, සෑම පරම්පරාවක්ම කරත යුතු